<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/Custom_Environment_for_Deep_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 61kB 3.9MB/s 


In [13]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [14]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [15]:
env = ShowerEnv()

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [16]:
env.observation_space.sample()

array([27.050766], dtype=float32)

In [17]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-60
Episode:2 Score:-4
Episode:3 Score:-52
Episode:4 Score:-44
Episode:5 Score:-44
Episode:6 Score:-60
Episode:7 Score:8
Episode:8 Score:-4
Episode:9 Score:-6
Episode:10 Score:12


In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [19]:
states = env.observation_space.shape
actions = env.action_space.n

In [20]:
actions

3

In [23]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
del model

In [33]:
model = build_model(states, actions)

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [35]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [36]:

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [37]:
del dqn

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 9:45 - reward: -1.0000

/usr/local/lib/python3.6/dist-packages/rl/memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 77s 8ms/step - reward: -0.6588
166 episodes - episode_reward: -39.855 [-60.000, 26.000] - loss: 1.961 - mae: 6.935 - mean_q: -7.668

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: -0.5434
167 episodes - episode_reward: -32.335 [-60.000, 44.000] - loss: 2.545 - mae: 11.078 - mean_q: -15.926

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 80s 8ms/step - reward: -0.4976
167 episodes - episode_reward: -29.832 [-60.000, 38.000] - loss: 2.932 - mae: 12.561 - mean_q: -18.168

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: -0.4460
166 episodes - episode_reward: -26.735 [-60.000, 34.000] - loss: 2.692 - mae: 12.046 - mean_q: -17.388

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: -0.4596
done, took 405.243 seconds


In [39]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -54.000, steps: 60
Episode 3: reward: -58.000, steps: 60
Episode 4: reward: -54.000, steps: 60
Episode 5: reward: -58.000, steps: 60
Episode 6: reward: -54.000, steps: 60
Episode 7: reward: -58.000, steps: 60
Episode 8: reward: -54.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -58.000, steps: 60
Episode 11: reward: -54.000, steps: 60
Episode 12: reward: -54.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -58.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -56.000, steps: 60
Episode 18: reward: -54.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -56.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -54.000, steps: 60
Episode 24: reward: -58.000, steps: 60
Episode 25: reward: -56.000, steps: 60
Episo

In [40]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: ignored

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')


In [41]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


TypeError: ignored